## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import re
from collections import Counter
from tqdm import tqdm

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

import ilm.ilm.tokenize_util
from ilm.ilm.infer import infill_with_ilm
from perturbation_functions import calculate_necc_and_suff, gen_num_samples_table, gen_probs_table

/users/eleves-b/2021/valentin.dorseuil/Desktop/SNLP_XAI_hate-speech/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2


## Loading the model 

In [10]:
# MODEL_DIR = './ilm/Models/ILM/'
# MASK_CLS = 'ilm.ilm.mask.hierarchical.MaskHierarchical'
# 
# tokenizer = ilm.ilm.tokenize_util.Tokenizer.GPT2
# with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
#     additional_ids_to_tokens = pickle.load(f)
# additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
# try:
#     ilm.ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
# except ValueError:
#     print('Already updated')
# print(additional_tokens_to_ids)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = GPT2LMHeadModel.from_pretrained(MODEL_DIR)


In [6]:
MODEL_DIR = './ilm/Models/ILM/'

# tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")



tokenizer = ilm.ilm.tokenize_util.Tokenizer.GPT2

with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
    additional_ids_to_tokens = pickle.load(f)
    print(additional_ids_to_tokens)
additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
try:
    ilm.ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
except ValueError:
    print('Already updated')
print(additional_tokens_to_ids)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

vocab_size = ilm.ilm.tokenize_util.vocab_size(tokenizer=tokenizer)

model.resize_token_embeddings(vocab_size)

model.eval()
_ = model.to(device)



{50257: '<|startofinfill|>', 50258: '<|endofinfill|>', 50259: '<|infill_document|>', 50260: '<|infill_paragraph|>', 50261: '<|infill_sentence|>', 50262: '<|infill_ngram|>', 50263: '<|infill_word|>'}
Already updated
{'<|startofinfill|>': 50257, '<|endofinfill|>': 50258, '<|infill_document|>': 50259, '<|infill_paragraph|>': 50260, '<|infill_sentence|>': 50261, '<|infill_ngram|>': 50262, '<|infill_word|>': 50263}


## Previous code

In [ ]:
# test_suite_cases = pd.read_csv("hatecheck-data/test_suite_cases.csv", index_col="case_id") #.drop(columns=['Unnamed:0'])
# test_suite_cases.drop(columns=['Unnamed: 0'], inplace=True)
# # target_ds = ['women', 'Muslims', 'men']
# funcs = ['derog_neg_emote_h', 'derog_neg_attrib_h', 'derog_dehum_h']
# 
# test_suite_cases = test_suite_cases[test_suite_cases.target_ident.isin(target_ds) & 
                                    # test_suite_cases.functionality.isin(funcs)]
# tts = [text for _, text in test_suite_cases.test_case.items()]
# 
# with open("Data/HateCheck_test_suite_cases.txt", "w") as f:
    # f.write("\n".join(tts))

## Adding men and catholic to the dataset

In [6]:
dic_men = {'SLUR_P_space_add' : 'men',
    'IDENTITY_S_space_add' : 'm e n',
    'IDENTITY_P_leet' : 'men',
    'IDENTITY_P' : 'men',
    'SLUR_S_leet' : 'boy',
    'SLUR_P_leet' : 'boys',
    'IDENTITY_A' : 'men',
    'IDENTITY_S' : 'man',
    'IDENTITY_P_char_del' : 'mn',
    'IDENTITY_P_space_add' : 'm e n',
    'SLUR_S_space_add' : 'b o y',
    'SLUR_S' : 'guy',
    'SLUR_P' : 'guys',
    'IDENTITY_S_char_del' : 'gu'
}


dic_catholics= {'SLUR_P_space_add' : 'Chrisian',
    'IDENTITY_S_space_add' : 'c a t h o l i c',
    'IDENTITY_P_leet' : 'Christians',
    'IDENTITY_P' : 'Catholics',
    'SLUR_S_leet' : 'Popist',
    'SLUR_P_leet' : 'Popists',
    'IDENTITY_A' : 'catholic',
    'IDENTITY_S' : 'Catholic',
    'IDENTITY_P_char_del' : 'Cathlics',
    'IDENTITY_P_space_add' : 'c a t h o l i c',
    'SLUR_S_space_add' : 'P o p i s t s',
    'SLUR_S' : 'Christian',
    'SLUR_P' : 'Christians',
    'IDENTITY_S_char_del' : 'Cathlic'
}


In [ ]:
def add_word_hatecheck(word_df: str, df: pd.DataFrame):
    indices = []
    for sentence in test_suite_cases[pd.notna(test_suite_cases['target_ident'])]['case_templ'].unique():
        index = int((df["case_templ"] == sentence).idxmax())
        indices.append(index)
        
        new_row = df.iloc[index].copy()

        match = re.search(r'\[(.*?)\]', sentence) #search in the case_templ sentence the part of the template to replace
        if match:
            to_replace = match.group(1)
        new_row['test_case']  = new_row['case_templ'].replace(f"[{to_replace}]", word_df[to_replace]) # replace with the right sentence 
        new_row['target_ident'] = word_df['IDENTITY_P'] #change the label

        df = pd.concat([df.iloc[:index+1], pd.DataFrame([new_row]), df.iloc[index+1:]]).reset_index(drop=True)
    
    return df


In [ ]:
test_suite_cases = pd.read_csv("hatecheck-data/test_suite_cases.csv", index_col="case_id") 
test_suite_cases.drop(columns=['Unnamed: 0'], inplace=True)


updated_df = add_word_hatecheck(dic_men, test_suite_cases)
updated_df = add_word_hatecheck(dic_catholics, updated_df)

test_suite_cases=updated_df

target_ds = ['women', 'Muslims', 'men', 'Catholics']
funcs = ['derog_neg_emote_h', 'derog_neg_attrib_h', 'derog_dehum_h']

test_suite_cases = test_suite_cases[test_suite_cases.target_ident.isin(target_ds) & 
                                    test_suite_cases.functionality.isin(funcs)]
tts = [text for _, text in test_suite_cases.test_case.items()]

with open("Data/HateCheck_test_suite_cases.txt", "w") as f:
    f.write("\n".join(tts))

## Generating perturbations

In [4]:
import contextlib
import io

In [7]:
# generate approximately 100 perturbations for each token. 
num_samples = gen_num_samples_table(20, 1)
probs_table = gen_probs_table(20)
mask_tokn = additional_tokens_to_ids['<|infill_ngram|>']

orig_texts = []
necc_perturbed = []
suff_perturbed = []
necc_masks = []
suff_masks = []

with open("Data/HateCheck_test_suite_cases.txt", "r") as ff:
    with tqdm(total=240) as pbar:
            for i, text in enumerate(ff):
                with contextlib.redirect_stdout(io.StringIO()):
                    necc_pp, suff_pp, necc_mm, suff_mm = calculate_necc_and_suff(text, ilm_tokenizer=tokenizer, ilm_model=model, cl_tokenizer=None, cl_model=None, num_samples=num_samples,
                                        mask_tokn=mask_tokn, additional_tokens_to_ids=additional_tokens_to_ids, probs_table=probs_table, 
                                        return_pert_only=True)

                orig_texts.append(text)
                necc_perturbed.append(necc_pp)
                suff_perturbed.append(suff_pp)
                necc_masks.append(necc_mm)
                suff_masks.append(suff_mm)
                pbar.update(1)

necc_suff_perturbations = {'orig_texts': orig_texts, 
                           'necc_perturbed': necc_perturbed, 
                           'suff_perturbed': suff_perturbed,
                           'necc_masks': necc_masks,
                           'suff_masks': suff_masks}

#pickle.dump(necc_suff_perturbations, open('Data/HateCheck_necc_suff_perturbations.pickle', 'wb'))
pickle.dump(necc_suff_perturbations, open('Data/HateCheck_necc_suff_perturbations_2.pickle', 'wb'))

  0%|          | 0/240 [01:15<?, ?it/s]


KeyboardInterrupt: 

## Saving perturbations

In [ ]:
with open("Data/hatecheck_perturbations/orig_texts.txt", "w") as ff:
    ff.write("\n".join(necc_suff_perturbations['orig_texts']))

In [ ]:
with open("Data/hatecheck_perturbations/necc_perturbations.tsv", "w") as ff:
    for ll in necc_suff_perturbations['necc_perturbed']:
        ff.write("\t".join(ll))
        ff.write("\n")

In [ ]:
with open("Data/hatecheck_perturbations/suff_perturbations.tsv", "w") as ff:
    for ll in necc_suff_perturbations['suff_perturbed']:
        ff.write("\t".join(ll))
        ff.write("\n")

In [ ]:
with open("Data/hatecheck_perturbations/necc_masks.tsv", "w") as ff: 
    for ll in necc_suff_perturbations['necc_masks']:
        llist = ll.astype(int).astype(str).tolist()
        ff.write("\t".join([" ".join(ii) for ii in llist]))
        ff.write("\n")

In [ ]:
with open("Data/hatecheck_perturbations/suff_masks.tsv", "w") as ff: 
    for ll in necc_suff_perturbations['suff_masks']:
        llist = ll.astype(int).astype(str).tolist()
        ff.write("\t".join([" ".join(ii) for ii in llist]))
        ff.write("\n")